In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import ADASYN
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("Cleaned_Lung_Cancer.csv")

In [ ]:
X = df.drop(columns=['survived'])
y = df['survived']

print("Class distribution in the target variable:")
print(y.value_counts(normalize=True))

adasyn = ADASYN(random_state=42)
X_train_balanced, y_train_balanced = adasyn.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_train_balanced, y_train_balanced, test_size=0.25, random_state=42)

class_weights = dict(zip([0,1], [1, (0.779771/0.220229)]))

rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weights)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

recall = recall_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")

print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred))

Class distribution in the target variable:
survived
0    0.779771
1    0.220229
Name: proportion, dtype: float64


In [ ]:
lr_model = LogisticRegression(random_state=42, class_weight=class_weights, max_iter=1000)
lr_model.fit(X_train, y_train)

y_lr_pred= lr_model.predict(X_test)
lr_cr=classification_report(y_test, y_lr_pred)

/home/bruno/Documents/AMS/survival_rate_prediction/venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
